# Confinement analysis WEST

In [ ]:
import logging
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import os
import pandas as pd
import seaborn as sns
import scipy
import scipy.io as sio
import scipy.interpolate as sc_interp
import scipy.signal as sc_sig
from scipy.optimize import curve_fit
import sklearn
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
print('Logging version:', logging.__version__)
print('Matplotlib version:', matplotlib.__version__)
print('NumPy version:', np.__version__)
print('Pandas version:', pd.__version__)
print('Seaborn version:', sns.__version__)
print('Scipy version:', scipy.__version__)
print('Sklearn version:', sklearn.__version__)

# Local modules
import imas
try:
    import imas_west
except ImportError as err:
    print(' ')
    print('WARNING: no module imas_west, warning:', err)
    print(' ')
try:
    import pywed as pw
except ImportError as err:
    print(' ')
    print('WARNING: no module pywed, warning:', err)
    print(' ')

## Read data

### C4

In [ ]:
pathfile = '/Imas_public/public/plateau_statistics/west/'
filename = 'reduced_dataBase_C4_WEST.h'

In [ ]:
stor = pd.HDFStore(pathfile+filename)

In [ ]:
stats = stor['stats']
sig   = stor['signals']
stor.close()

In [ ]:
stats.shape

In [ ]:
stats.tail()

## Example: computation radiated fraction and P_rad divertor

In [ ]:
stats['P_radDiv_mean_plto']  = stats['P_rad_mean_plto'] - stats['P_radBulk_mean_plto']

In [ ]:
stats['f_rad_mean_plto']     = stats['P_rad_mean_plto'] / stats['P_TOT_mean_plto']
stats['f_radBulk_mean_plto'] = stats['P_radBulk_mean_plto'] / stats['P_TOT_mean_plto']
stats['f_radDiv_mean_plto']  = stats['P_radDiv_mean_plto'] / stats['P_TOT_mean_plto']

In [ ]:
stats[['P_radDiv_mean_plto', 'f_rad_mean_plto', 'f_radBulk_mean_plto', 'f_radDiv_mean_plto', 'te_mean_plto', 'ne3_mean_plto']].describe()

In [ ]:
sig['P_radDiv'] = sig['P_rad'] - sig['P_radBulk'] 

In [ ]:
sig['f_rad']     = sig['P_rad'] / sig['P_TOT']
sig['f_radBulk'] = sig['P_radBulk'] / sig['P_TOT']
sig['f_radDiv']  = sig['P_radDiv'] / sig['P_TOT']

In [ ]:
shot_plt = 54719
sns.set('talk', font_scale=1.2, rc={'figure.figsize':(17, 5.5)})
plt.subplots_adjust(wspace=0.15, hspace=0.)
plt.subplot(1, 2, 1)
plt.plot(sig['time'][shot_plt], sig['f_rad'][shot_plt], 'r', label='f_rad')
plt.plot(sig['time'][shot_plt], sig['f_radBulk'][shot_plt], 'b', label='f_radBulk')
plt.plot(sig['time'][shot_plt], sig['f_radDiv'][shot_plt], 'g', label='f_radDiv')
plt.ylim((0, 1))
plt.legend();
plt.subplot(1, 2, 2)
plt.plot(sig['time'][shot_plt], 1.E-6*sig['P_TOT'][shot_plt], label='P_TOT')
for ii in range(len(stats['P_TOT_mean_plto'][shot_plt])):
    plt.plot(np.linspace(stats['tIni_plto'][shot_plt][ii], \
                         stats['tEnd_plto'][shot_plt][ii], 10),
             np.linspace(1.E-6*stats['P_TOT_mean_plto'][shot_plt][ii], \
                         1.E-6*stats['P_TOT_mean_plto'][shot_plt][ii], 10), \
            linewidth=5, label='plateau '+str(ii))
plt.ylim((0, 6))
plt.legend();

## Filter data

In [ ]:
stats = stats[(stats['eq_q_95_mean_plto'] < 20.) \
              & (stats['P_COND_mean_plto'] > 0.) \
              & (stats['f_rad_mean_plto'] > 0.26) \
              & (stats['eq_w_mhd_mean_plto'] > 0) \
              & ((stats['shot'] < 55625) | (stats['shot'] > 55643))]# & (stats['te_mean_plto'] > -1.E10) & (stats['ne3_mean_plto'] > -1.E10)]

## First shot after boronisation

In [ ]:
after_boro_shot = [53453, 54288, 54403, 54502, 54596, 54719, 54881, 55000, 55138, 55499, 55548, 55747, 55795]

In [ ]:
boro_shot_distance     = np.full(stats['shot'].size, np.nan)
boro_shot_distance_all = np.full(len(after_boro_shot), np.nan)
for jj in range(stats['shot'].size):
    for ii in range(len(after_boro_shot)):
        boro_shot_distance_all[ii] = stats['shot'][jj] - after_boro_shot[ii]
    if (~np.all(boro_shot_distance_all < 0) and ~np.all(np.isnan(boro_shot_distance_all))):
        try:
            boro_shot_distance[jj] = np.nanmin(boro_shot_distance_all[boro_shot_distance_all >= 0])
        except:
            print(jj)
            print(stats['shot'][jj])
            print(boro_shot_distance_all)
            raise
stats['boro_shot_distance'] = boro_shot_distance

In [ ]:
plt.scatter(stats['shot'], stats['boro_shot_distance']);

## Helium shots

In [ ]:
helium_shots = [(55230, 55498), (55827, 55987)]

In [ ]:
for iishot in helium_shots:
    #print(iishot[0])
    stats = stats[(stats['shot'] < iishot[0]) | (stats['shot'] > iishot[1])]

In [ ]:
stats.shape